<a href="https://colab.research.google.com/github/shu0518/hw2_m11423036/blob/main/%E5%AF%A6%E9%A9%97%E4%B8%80/SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

# -----------------------------
# 1. 讀取資料
# -----------------------------
# 抓本地資料
# train_path = "adult.data"
# test_path = "adult.test"
# 抓github 資料比較耗時
train_path = "https://raw.githubusercontent.com/shu0518/hw2_m11423036/refs/heads/main/adult/adult.data"
test_path  = "https://raw.githubusercontent.com/shu0518/hw2_m11423036/refs/heads/main/adult/adult.test"

column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income"
]

train_df = pd.read_csv(train_path, header=None, names=column_names, sep=",", engine="python")
test_df  = pd.read_csv(test_path,  header=None, names=column_names, sep=",", engine="python")

# 去除空白
train_df = train_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
test_df  = test_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# 確認資料集是否讀取成功並顯示前5筆和大小
print("--- 訓練資料集前 5 筆 ---")
display(train_df.head())
print(f"訓練資料集大小: {train_df.shape}")

print("\n--- 測試資料集前 5 筆 ---")
display(test_df.head())
print(f"測試資料集大小: {test_df.shape}")

# For faster experimentation, you can reduce the dataset size:
# train_df = train_df.sample(frac=0.1, random_state=42)
# test_df = test_df.sample(frac=0.1, random_state=42)

# -----------------------------
# 2. 修正數值欄位
# -----------------------------
numeric_cols_all = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

for df in [train_df, test_df]:
    for col in numeric_cols_all:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 移除 rows with invalid number
train_df = train_df.dropna(subset=["hours_per_week", "age"])
test_df  = test_df.dropna(subset=["hours_per_week", "age"])

# -----------------------------
# 3. X / y
# -----------------------------
target_col = "hours_per_week"

X_train = train_df.drop(columns=[target_col, "income"])
y_train = train_df[target_col].astype(float)

X_test = test_df.drop(columns=[target_col, "income"])
y_test = test_df[target_col].astype(float)

X_train = X_train.replace("?", np.nan)
X_test  = X_test.replace("?", np.nan)

numeric_features = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss"]
categorical_features = [c for c in X_train.columns if c not in numeric_features]

# -----------------------------
# 4. Preprocess
# -----------------------------
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# -----------------------------
# 5. 定義模型
# -----------------------------
models = {
    "SVR_rbf": SVR(kernel="rbf", C=10.0, epsilon=0.1),
}

results = []

# -----------------------------
# 6. 訓練 + 預測 + 評估
# -----------------------------
for name, model in models.items():
    print("\n============================")
    print(f"訓練模型：{name}")
    print("============================")

    clf = Pipeline([
        ("preprocess", preprocess),
        ("model", model)
    ])

    # 訓練時間
    t0 = time.perf_counter()
    clf.fit(X_train, y_train)
    train_time = time.perf_counter() - t0

    # 預測時間
    t0 = time.perf_counter()
    y_pred = clf.predict(X_test)
    pred_time = time.perf_counter() - t0

    # 評估（修正 RMSE 計算方式）
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    r2   = r2_score(y_test, y_pred)

    print(f"RMSE : {rmse:.4f}")
    print(f"MAPE : {mape:.2f}%")
    print(f"R²   : {r2:.4f}")
    print(f"訓練時間: {train_time:.4f} 秒")
    print(f"預測時間: {pred_time:.4f} 秒")

    results.append({
        "model": name,
        "RMSE": rmse,
        "MAPE(%)": mape,
        "R2": r2,
        "train_time(s)": train_time,
        "predict_time(s)": pred_time
    })


/tmp/ipython-input-835213743.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_df = train_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/tmp/ipython-input-835213743.py:38: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_df  = test_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


--- 訓練資料集前 5 筆 ---


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


訓練資料集大小: (32561, 15)

--- 測試資料集前 5 筆 ---


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


測試資料集大小: (16281, 15)

訓練模型：SVR_rbf
RMSE : 10.9187
MAPE : 30.16%
R²   : 0.2344
訓練時間: 206.7850 秒
預測時間: 28.5254 秒
